# Object Detection using YOLO V2
# Trained using COCO Dataset - 80 classes

In [ ]:
# import the needed modules
import os
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
from PIL import Image

from keras import backend as K
from keras.models import load_model

# The below provided fucntions will be used from yolo_utils.py
from yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image, draw_boxes

# The below functions from the yad2k library will be used
from keras_yolo import yolo_head, yolo_eval

## Load Pre-trained Model

In [ ]:
# Loading the classes and the anchor boxes
class_names = read_classes("weights/yolov2_classes.txt")
anchors = read_anchors("weights/yolov2_anchors.txt")

# Load the pretrained model
yolo_model = load_model("weights/yolov2.h5")

# Convert final layer features to bounding box parameters
yolo_outputs = yolo_head(yolo_model.output, anchors, len(class_names))

In [ ]:
# Print the summary of the model
yolo_model.summary()

## Load Test Image

In [ ]:
# load the image to be fed through the network
input_image_name = "data/" + "test.jpg"
input_image = Image.open(input_image_name)

# Show input image
imshow(input_image)

In [ ]:
# Obtaining the dimensions of the input image
width, height = input_image.size
width = np.array(width, dtype=float)
height = np.array(height, dtype=float)

# Assign the shape of the input image to image_shapr variable
image_shape = (height, width)

In [ ]:
# Preprocess the input image before feeding into the convolutional network
image, image_data = preprocess_image(input_image_name, model_image_size=(608, 608))

# Evaluate Test Image

In [ ]:
# yolo_eval function selects the best boxes using filtering and 
# non-max suppression techniques.
# scale yolo_outputs to test image size
boxes, scores, classes = yolo_eval(yolo_outputs, image_shape)

In [ ]:
# Initiate a session
sess = K.get_session()

# Run the session
out_scores, out_boxes, out_classes = sess.run([scores, boxes, classes], 
                                              feed_dict={
                                                  yolo_model.input: image_data,
                                                  K.learning_phase(): 0
                                              })

# Print the results
print('Found {} boxes for {}'.format(len(out_boxes), input_image_name))

In [ ]:
# Produce the colors for the bounding boxs
colors = generate_colors(class_names)

# Draw the bounding boxes
draw_boxes(image, out_scores, out_boxes, out_classes, class_names, colors)

# Show output image
imshow(image)